Lets see what we can do with a wordmap (plot, noise reduction and possibly partitioning into morphemes?)

In [193]:
import wordmapper
import text_utilities as tu
import numpy as np # arrays
import scipy.stats as stats # arithmetics
import matplotlib.pyplot as pp # plotting

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
pc = tu.PosCorpus('../data/experiment/verbs/joined/')

targets = "beschissen,stehen,anschauen,viertelt,verunglimpfst"
wm_mt = []
for t in targets.split(","):
    wm = wordmapper.WordMapper(t, pc.counted_corpus)
    mt = wordmapper.MapToken(t, wm.wordmap, pc.metrics)
    wm_mt.append((mt.token, mt.wordmap))

TypeError: __init__() takes 3 positional arguments but 4 were given

In [109]:
def plot_targets(maptokens, absolute=False, zscore=False, med=False, std=False, mean=False, mmmix=False, zeroline=False, derive=False):
    for (target, wm) in maptokens:

        arr = np.array(wm)
        wm_med = np.array([np.median(wm) for i in wm])
        wm_mea = np.array([np.mean(wm) for i in wm])
        wm_std = np.array([np.std(wm) for i in wm])
        wm_mix = [(x+y)/2 for x, y, in zip(wm_mea, wm_med)]
        wm_zero = [0 for i in range(len(arr))]
        wm_derive = wm_derive = np.array(stats.zscore(tu.derive_wordmap(wm)))
        wm_zscore = np.array(stats.zscore(arr))

        if absolute:
            pp.plot(range(len(arr)), arr)
        if med:
            pp.plot(range(len(arr)), wm_med)
        if mean:
            pp.plot(range(len(arr)), wm_mea)
        if std:
            pp.plot(range(len(arr)), wm_std)
        if mmmix:
            pp.plot(range(len(arr)), wm_mix)
        if zscore:
            pp.plot(range(len(arr)), wm_zscore)
        if zeroline:
            pp.plot(range(len(arr)), wm_zero)
        if derive:
            pp.plot(range(len(arr)), wm_derive)


    pp.xlabel('Character positions')
    pp.ylabel('Frequency')
    pp.xticks(np.arange(stop=len(arr),step=1), labels=list(target))
    pp.savefig("/home/gnom/Pictures/wordmaps/zscore" + target + ".png")
    pp.clf()

In [113]:
for (x,y) in wm_mt:
    plot_targets(
        maptokens=[(x,y)],
        absolute=False,
        med=  False,
        mmmix=False,
        mean= False,
        zscore=  True,
        zeroline=True,
        derive=  True
    )

<Figure size 640x480 with 0 Axes>

0.6054367328205298


In [211]:
from tqdm import tqdm as tq

lex_morphemes = []
fun_morphemes = []


for k in pc.counted_corpus:
    for v in tq(pc.counted_corpus[k]):
        wm = wordmapper.WordMapper(v, pc.counted_corpus)
        mt = wordmapper.MapToken(v, wm.wordmap)
        lex_morphemes.append(mt.stem)
        fun_morphemes.extend(mt.affix)

#with open("../new_tokenizer/lex_vocab.txt", encoding="utf8", mode="w") as lv:

100%|██████████| 32/32 [00:20<00:00,  1.59it/s]


In [276]:
with open("../new_tokenizer/lex_vocab_raw.txt", encoding="utf8", mode="w") as lv:
    lv.write("".join(s + "\n" for s in lex_morphemes))
    lv.close()

with open("../new_tokenizer/fun_vocab_raw.txt", encoding="utf8", mode="w") as fv:
    fv.write("".join([s + "\n" for s in fun_morphemes]))
    fv.close()

CONVERT FUNCTIONAL MORPHEME MAP TO HASHABLE WITH RELATIVE FREQUENCIES

In [232]:
import collections as cl

In [266]:
fm_clean = [i for i in fun_morphemes if i != ""]  # select non-empty morphemes
fm_ncount = cl.Counter(fm_clean).most_common()
n_o_fm = len(fm_clean)
fm_rel = {k:v/n_o_fm if len(k)>1 else 0 for k, v in fm_ncount } # unary morphemes get no weight
fm_rel

{'t': 0,
 'e': 0,
 'st': 0.10979939410248175,
 'te': 0.10571088792632008,
 'a': 0,
 'h': 0,
 'en': 0.02912301108497322,
 'v': 0,
 'b': 0,
 'ver': 0.018081440265275476,
 'z': 0,
 's': 0,
 'g': 0,
 'ier': 0.016605758630567444,
 'zu': 0.013663075841355544,
 'd': 0,
 'er': 0.012439128132568294,
 'be': 0.011041570819697746,
 'l': 0,
 'rt': 0.008333260995998298,
 'f': 0,
 'r': 0,
 'n': 0,
 'el': 0.006710011197819463,
 'k': 0,
 'au': 0.00617182142516124,
 'ch': 0.005946128939852953,
 're': 0.005590229251482192,
 'ert': 0.005477383008828048,
 'm': 0,
 'her': 0.004939193236169825,
 'w': 0,
 'ge': 0.004105867136569995,
 'ab': 0.0036458016857492557,
 'an': 0.003437470160849298,
 'hin': 0.0030555290318660428,
 'ere': 0.002960043749620229,
 'u': 0,
 'p': 0,
 'ein': 0.0026735879028827875,
 'we': 0.002413173496757841,
 'um': 0.002309007734307862,
 'sch': 0.0022395638926745428,
 'über': 0.002118037169816234,
 'ß': 0,
 'ck': 0.0016753326794038245,
 'lt': 0.00164929123879133,
 'ss': 0.001614569317974670

LEXEMIC MORPHEMES

In [274]:
lm_clean = [i for i in lex_morphemes if len(i)>1]  # select only morphemes longer than 1 character
lm_ncount = cl.Counter(lm_clean).most_common()
n_o_lm = len(lm_clean)
lm_rel = {k:v/n_o_lm if len(k)>1 else 0 for k, v in lm_ncount } # unary morphemes get no weight
lm_rel

{'schrei': 0.0003197953309881676,
 'schnei': 0.0002942117045091142,
 'ei': 0.00028141989126958747,
 'au': 0.00028141989126958747,
 'rei': 0.00023025263831148065,
 'schie': 0.00023025263831148065,
 'lie': 0.00019187719859290055,
 'lau': 0.00019187719859290055,
 'wuch': 0.00019187719859290055,
 'wach': 0.00017908538535337383,
 'schla': 0.00017908538535337383,
 'ar': 0.00017908538535337383,
 'rau': 0.00017908538535337383,
 'wir': 0.00016629357211384713,
 'schau': 0.00016629357211384713,
 'stür': 0.00016629357211384713,
 'al': 0.00016629357211384713,
 'ie': 0.00016629357211384713,
 'grei': 0.00015350175887432043,
 'hau': 0.00015350175887432043,
 'schlie': 0.00015350175887432043,
 'blie': 0.00015350175887432043,
 'strei': 0.00015350175887432043,
 'blei': 0.00015350175887432043,
 'brau': 0.00015350175887432043,
 'in': 0.00015350175887432043,
 'wand': 0.00014070994563479373,
 'frühstück': 0.00014070994563479373,
 'flie': 0.00014070994563479373,
 'bekomm': 0.00014070994563479373,
 'schlaf': 0.

In [273]:
[i for i in lm_clean if i.startswith("sch")]

['schwindl',
 'schnöker',
 'schleuder',
 'schleuder',
 'schlagzeil',
 'scheuch',
 'schlagnahm',
 'schmutz',
 'schlechter',
 'schlosser',
 'schmiss',
 'schütte',
 'schloss',
 'schling',
 'schücht',
 'schrei',
 'schluder',
 'schlotter',
 'schüttgeh',
 'schmäler',
 'schild',
 'schreck',
 'scherbel',
 'schrumpel',
 'schmurgel',
 'schnüff',
 'schnauf',
 'schmölz',
 'schände',
 'schott',
 'schnüff',
 'schoss',
 'schob',
 'schloss',
 'schädi',
 'schwefel',
 'schütt',
 'schmach',
 'schlacker',
 'schlimmer',
 'schnüffel',
 'schädig',
 'schaff',
 'schor',
 'schrieb',
 'schränk',
 'schaud',
 'schlu',
 'schücht',
 'schuster',
 'schwäbel',
 'schnupper',
 'schreib',
 'schüchter',
 'schlagnahm',
 'schaff',
 'schlank',
 'schrie',
 'schattier',
 'schwamm',
 'schmalz',
 'schwatz',
 'schnörkel',
 'schachte',
 'schweig',
 'schmetter',
 'schlüp',
 'schiss',
 'schlag',
 'scheffel',
 'schnüffe',
 'schätztho',
 'schuld',
 'schrumpl',
 'schmett',
 'schnitt',
 'schwäger',
 'schlimmer',
 'schien',
 'schied',
 's

True